__Obiettivo__

Classificare gli articoli scientifici in base ad una lista di _topic_ predefinita.

Il notebook implementa una __classificazione multi-label__ di un insieme di paper accademici in base ad una lista di _topic_ precedentemente stilata, riportata all'interno del path __../markdown/topics.md__.
Per riuscire nell'intento, è stata attuata una fase preliminare di __preprocessing__, in cui le operazioni principali si suddividono in:
- Conversione del testo in minuscolo
- Suddivisione in _token_ del periodo analizzato
- Rimozione di numeri e punteggiatura ove necessario
- Rimozione delle _stopwords_, rese disponibili tramite la libreria __nltk__
- __Lemmatizzazione__ di ogni _token_ che costituisca il testo, ossia conversione del termine circoscritto nella sua forma canonica

_(Descrivere poi dei risultati ottenuti, magari dei grafici e cosi via)_

In [20]:
import nltk

from nltk.corpus import stopwords

nltk.download("wordnet")
nltk.download("stopwords")
set_stopwords = set(stopwords.words("english"))

[nltk_data] Downloading package wordnet to /home/matte/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/matte/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [21]:
from typing import Dict

class Paper:
    def __init__(self, title, abstract):
        self.title = title
        self.abstract = abstract 

    def get_dict(self) -> Dict[str, str]:
        return {
            "Title": self.title,
            "Abstract": self.abstract
        }

In [22]:
import regex

from typing import List, Set

def define_keywords() -> Dict[str, Set[str]]:
    dict_keywords: Dict[str, Set[str]] = {}

    with open("../markdown/topics.md", "r") as file:
        content = file.read()
        
    categories = regex.findall(r"### (\d+)\.", content)
    for category in categories:
        matches = regex.findall(r"{}\.[0-9]?(.*\w)".format(category), content)
        cleaned_matches = [match.strip().replace("*", "") for match in matches]

        _set = set()
        for match in cleaned_matches[1:]:
            _set.add(match)

        dict_keywords[cleaned_matches[0]] = _set

    return dict_keywords

keywords = define_keywords()

In [25]:
import json

from pandas import DataFrame
from langdetect import detect
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from deep_translator import GoogleTranslator

def get_json() -> List[Dict]:
    with open("../json/grobid/summary.json", "r") as file:
        content = file.read()

    return json.loads(content)

lemmatizer = WordNetLemmatizer()
tokenizer = RegexpTokenizer(r"[a-zA-Z]+")

def preprocess_field(field: str) -> List[str] | None:
    try:
        if detect(field) != "en":
            field = GoogleTranslator(source="auto", target="en").translate(text=field)

        # Converting field to lower case and remove numbers and dots
        text = field.lower()
        tokens = tokenizer.tokenize(text)

        # Removing words less than two characters and stopwords
        tokens = [token for token in tokens if len(token) > 2]
        tokens = [token for token in tokens if token not in set_stopwords]

        # Applying lemmatizer for each paper's word
        return [lemmatizer.lemmatize(token) for token in tokens]
    except Exception:
        return None

def define_papers_dict(_json: List[Dict]) -> Dict[str, Paper]:
    dict_papers: Dict[str, Paper] = {}

    for _dict in _json:
        for key, value in _dict.items():
            title = preprocess_field(value["Title"])
            abstract = preprocess_field(value["Abstract"])

            # Before the assigning the fields will be preproccessed
            dict_papers[key] = Paper(title, abstract)

    return dict_papers

_json = get_json()
dict_papers = define_papers_dict(_json)

def define_papers_dataframe(_dict: Dict[str, Paper]) -> DataFrame:
    records = []

    for key, value in _dict.items():
        try:
            record = {
                "Name": key,
                "Title": " ".join(value.title),
                "Abstract": " ".join(value.abstract) 
            }
 
            records.append(record)
        except Exception:
            continue
    
    return DataFrame(records)

df_papers = define_papers_dataframe(dict_papers)

Possiedo:
- Le keyword prese dai topic di Di Iorio
- Un dataframe composto da (Name, Title, Abstract) a cui ho applicato preprocessing

Ora manca la classificazione.